In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys, os
import glob
import pydicom
ROOT_DIR = '/home/gagliardi/rnsa'
sys.path.append(os.path.join(ROOT_DIR, 'SSD-Tensorflow'))  # To find local version of the library
from nets import ssd_vgg_512, ssd_common, np_methods
import tensorflow as tf
import parse_tfrecord
import numpy as np
from preprocessing import ssd_vgg_preprocessing
slim = tf.contrib.slim

In [3]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

# Iterator

In [4]:
filenames = glob.glob('/home/gagliardi/rnsa/data/tf_record_val_data/*')
iterator = parse_tfrecord.get_iterator(filenames)
filename, image, labels, bboxes = iterator.get_next()

In [5]:
# with queues.QueueRunners(sess):# with q 
# Start populating queues.
coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord)

# Test SSD-512 model using TFRecords pipeline

In [6]:
ckpt_filename = './logdir/ssd_512.ckpt'

In [12]:
# SSD object.
reuse = True if 'ssd' in locals() else None
params = ssd_vgg_512.SSDNet.default_params
ssd = ssd_vgg_512.SSDNet(params)

# Image pre-processimg
out_shape = ssd.params.img_shape

data_format = 'NHWC'

image.set_shape([None, None, 3])

# Evaluation pre-processing: resize to SSD net shape.
image_pre, labels_pre, bboxes_pre, bbox_img = ssd_vgg_preprocessing.preprocess_for_eval(image, 
                                                                                        None, 
                                                                                        None, 
                                                                                        out_shape, 
                                                                                        data_format, 
                                                                                        resize=ssd_vgg_preprocessing.Resize.WARP_RESIZE)
image_4d = tf.expand_dims(image_pre, 0)

with slim.arg_scope(ssd.arg_scope(weight_decay=0.0005)):
    predictions, localisations, logits, end_points = ssd.net(image_4d, is_training=False, reuse=reuse)

layers_anchors = ssd.anchors(out_shape, dtype=np.float32)

saver = tf.train.Saver()

saver.restore(sess, ckpt_filename)

INFO:tensorflow:Restoring parameters from ./logdir/ssd_512.ckpt


In [9]:
# Targets encoding.
target_labels, target_localizations, target_scores = ssd_common.tf_ssd_bboxes_encode(labels, 
                                                                                     bboxes_pre, 
                                                                                     layers_anchors, 
                                                                                     num_classes=params.num_classes, 
                                                                                     no_annotation_label=params.no_annotation_label)

In [19]:
# Output decoding.
localisations = ssd.bboxes_decode(localisations, layers_anchors)
tclasses, tscores, tbboxes = ssd_common.tf_ssd_bboxes_select(predictions, localisations, params.num_classes)

ValueError: slice index 2 of dimension 2 out of bounds. for 'ssd_bboxes_select_5/ssd_bboxes_select_layer/strided_slice_1' (op: 'StridedSlice') with input shapes: [1,16384,2], [3], [3], [3] and with computed input tensors: input[1] = <0 0 2>, input[2] = <0 0 3>, input[3] = <1 1 1>.

In [14]:
tclasses, tscores, tbboxes = ssd_common.tf_ssd_bboxes_select(predictions, localisations)

tclasses, tscores, tbboxes = ssd_common.tf_bboxes_sort(tclasses, tscores, tbboxes, top_k=400)

tclasses, tscores, tbboxes = ssd_common.tf_bboxes_nms_batch(tclasses, tscores, tbboxes, nms_threshold=0.3, num_classes=ssd.params.num_classes)

ValueError: slice index 2 of dimension 2 out of bounds. for 'ssd_bboxes_select_2/ssd_bboxes_select_layer/strided_slice_1' (op: 'StridedSlice') with input shapes: [1,16384,2], [3], [3], [3] and with computed input tensors: input[1] = <0 0 2>, input[2] = <0 0 3>, input[3] = <1 1 1>.

In [ ]:
sess.run(iterator.initializer)

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)

In [ ]:
_rpredictions, _rlocalisations = sess.run([rpredictions, rlocalisations])

In [ ]:
_rlocalisations[1].shape

In [ ]:
# SSD default anchor boxes.
ssd_anchors = ssd_net.anchors(net_shape)


rclasses, rscores, rbboxes = np_methods.ssd_bboxes_select(rpredictions, 
                                                         rlocalisations, 
                                                         ssd_anchors, 
                                                         select_threshold=0.5, 
                                                         img_shape=net_shape, 
                                                         num_classes=2, 
                                                         decode=True)
rbboxes = np_methods.bboxes_clip(rbbox_img, rbboxes)
rclasses, rscores, rbboxes = np_methods.bboxes_sort(rclasses, rscores, rbboxes, top_k=400)
rclasses, rscores, rbboxes = np_methods.bboxes_nms(rclasses, rscores, rbboxes, nms_threshold=.45)
# Resize bboxes to original image shape. Note: useless for Resize.WARP!
rbboxes = np_methods.bboxes_resize(rbbox_img, rbboxes)
saver = tf.train.Saver()
ckpt_filename = './logdir/ssd_512.ckpt'
saver.restore(sess, ckpt_filename)

In [ ]:
# Use the saver object normally after that.
dict_results = {}

sess.run(iterator.initializer)

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(sess=sess, coord=coord)


for i in range(566):
    filename_, image_4d_, rclasses_, rscores_, rbboxes_ = sess.run([filename, image_4d, rclasses, rscores, rbboxes])
    dict_results[filename_]=(image_4d_, rclasses_, rscores_, rbboxes_)